In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from ast import literal_eval
warnings.filterwarnings("ignore")

In [ ]:

df=pd.read_csv("./r/movies_metadata.csv")
dfkeyword = pd.read_csv('./r/keywords.csv')
dflinks =pd.read_csv("./r/links.csv")
dfcredits =pd.read_csv('./r/credits.csv')
dfratings = pd.read_csv('./r/ratings_small.csv')
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df["vote_count"] = df['vote_count'].notnull().astype('int')
df['vote_average'] = df['vote_average'].notnull().astype('int')


In [ ]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')

In [ ]:
df
dfkeyword

In [ ]:
df

In [ ]:
data = dfratings

average_ratings = data.groupby('id')['rating'].mean().reset_index()
average_ratings.columns = ['id', 'average_rating']

# Merge the average ratings with the original data
merged_data = pd.merge( average_ratings,data, on='id')

In [ ]:
main = pd.merge(df, dfkeyword, 'left')
main1 = pd.merge(main, dfcredits, 'outer')
main2 = pd.merge(main1, dflinks, 'outer')

main3 = pd.merge(main2, merged_data, "outer", left_on="id", right_on="id")
main3.head()

In [ ]:
merged_data

In [ ]:
#Understanding the data
df.shape

In [ ]:
# Basics Stats summary of my data
df.describe(include="all")

In [ ]:
df.info() # general info of my whole data

In [ ]:
dfkeyword.info()

In [ ]:
 # No Dulplicate values

In [ ]:
df.isnull().sum() # Checking for null values

In [ ]:
# total uniques values in each columns

In [ ]:
df["recommendation"] = ""

Data Preprocessing

In [ ]:
df.head()

In [ ]:
df['recommendation']

Exploratory Data Analysis(EDA)

Univariate Analysis

In [ ]:

XM = main3.drop("average_rating",axis=1)
XMz = XM.drop("imdbId", axis=1).drop("tmdbId", axis=1)
XMzx = XMz.drop("timestamp", axis=1)
Xa = XMzx.dropna(axis=0, subset=['id',"title"])

X = Xa.drop("id", axis=1)
y = Xa["id"]
Xa


In [ ]:
X["genres"] = X['genres'].astype("str")
X

In [ ]:
y

Modelling

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [ ]:
X_train


In [ ]:
X_test

In [ ]:
y_train

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from itertools import combinations
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.tree import export_graphviz

In [ ]:
class Node:
    def __init__(self, feature_index=None, threshold=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.value = value
        self.left = None
        self.right = None
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    def _calculate_mae(self, y):
        return np.mean(np.abs(y - np.mean(y)))
    def _split_dataset(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]
    def _find_best_split(self, X, y):
        best_mae = np.inf
        best_feature_index = None
        best_threshold = None
        for feature_index in range(X.shape[1]):
            unique_values = np.unique(X[:, feature_index])
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split_dataset(X, y, feature_index, threshold)
                mae = (len(y_left) * self._calculate_mae(y_left) + len(y_right) * self._calculate_mae(y_right)) / len(y)
                if mae < best_mae:
                    best_mae = mae
                    best_feature_index = feature_index
                    best_threshold = threshold
        return best_feature_index, best_threshold
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(y) < self.min_samples_split:
            return Node(value=np.mean(y))
        feature_index, threshold = self._find_best_split(X, y)
        if feature_index is None or threshold is None:
            return Node(value=np.mean(y))
        X_left, y_left, X_right, y_right = self._split_dataset(X, y, feature_index, threshold)
        node = Node(feature_index=feature_index, threshold=threshold)
        node.left = self._build_tree(X_left, y_left, depth + 1)
        node.right = self._build_tree(X_right, y_right, depth + 1)
        return node
    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)
    def predict(self, X):
        return [self._traverse_tree(x, self.root) for x in X]

In [ ]:
print('X_train columns:', X_train.columns)
print('X_test columns:', X_test.columns)

# Check the shape of X_train and X_test
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)


In [ ]:
X

In [ ]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first',  handle_unknown='ignore'),[1,3,4,5,6,7,8])
],remainder='passthrough')

step2 = DecisionTreeRegressor(max_depth=6)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)


y_pred = pipe.predict(X_test)

# Calculate evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the evaluation metricsx
print('R2 score:', r2)
print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)

exporting the model

In [ ]:
import pickle

pickle.dump(df,open('df.pkl','wb')) # we need this to create dropdowns in the website
pickle.dump(pipe,open('pipe.pkl','wb')) # Using the random forest model as its the highest